In [1]:
from psipy.model import MASOutput
from psipy.data import sample_data
import matplotlib.pyplot as plt
import os,numpy as np
from parfive import Downloader
from scipy.interpolate import interp2d,RegularGridInterpolator as rgi
dl = Downloader()

CRN = 2259

def get_cr_url(CRN):
    if CRN < 2241 :  
        return f"https://www.predsci.com/data/runs/cr{CRN}-medium"
    else : 
        return f'https://www.predsci.com/data/runs/cr{CRN}-high'

local_dir = f'/media/sam/6165-6433/MAS_Models/{CRN}'
if not os.path.isdir(local_dir) : 
    os.mkdir(local_dir)
    os.mkdir(local_dir+'/corona')
    os.mkdir(local_dir+'/helio')
local_path = local_dir+'/{vartype}/{var}002.hdf'
heliopath = f'{get_cr_url(CRN)}/hmi_masp_mas_std_0201/helio/''{var}002.hdf'
coronapath = f'{get_cr_url(CRN)}/hmi_masp_mas_std_0201/corona/''{var}002.hdf'

for var in ['rho', 'vr', 'br',
            'bt','bp','vt','vp',
            'p','jr','jt','jp','t','cs','va'
           ]:
#     dl.enqueue_file(coronapath.format(var=var), 
#                     path=local_path.format(var=var)
#                    )
    if not os.path.exists(local_path.format(vartype='corona',var=var)) :
        wget_cmd = (f"wget -O {local_path.format(vartype='corona',var=var)} " +
                   f"{coronapath.format(var=var)}" +
                    " --no-check-certificate"
        )
        print(wget_cmd)
        os.system(wget_cmd)
    if not os.path.exists(local_path.format(vartype='helio',var=var)) :
        wget_cmd = (f"wget -O {local_path.format(vartype='helio',var=var)} " +
                   f"{heliopath.format(var=var)}" +
                    " --no-check-certificate"
        )
        print(wget_cmd)
        os.system(wget_cmd)


ModuleNotFoundError: No module named 'psipy'

In [2]:
corona_model = MASOutput(local_path.format(vartype='corona',
                                      var='')[:-7]
                   );cbr= corona_model['br']
helio_model =  MASOutput(local_path.format(vartype='helio',
                                      var='')[:-7]
                   )

NameError: name 'MASOutput' is not defined

In [3]:
def do_lon_slice(idx = 0,rmax=30,rasterth = 4, rasterr=8) :
    cbr= corona_model['br']
    cbn = corona_model['bp']

    cbr_lonslice = cbr.data[idx,:,:,0]
    cbn_lonslice = cbn.data[idx,:,:,0]

    ## Interpolate to same coords 
    cbn_interpolated = interp2d(cbn.r_coords,
                                cbn.theta_coords,
                                cbn_lonslice)(cbr.r_coords,cbr.theta_coords)

    rmesh,thmesh = np.meshgrid(cbr.r_coords,cbr.theta_coords)

    cbx = np.array(cbr_lonslice)*np.cos(thmesh) - cbn_interpolated*np.sin(thmesh)
    cby = np.array(cbr_lonslice)*np.sin(thmesh) + cbn_interpolated*np.cos(thmesh)

    fig=plt.figure(figsize=(10,10))
    ax=fig.add_subplot(projection='polar')
    ax.contourf(
        cbr.theta_coords,
        cbr.r_coords,
        np.log(np.sqrt(cbr_lonslice**2 + 
                cbn_interpolated**2)).T,
        cmap='Greys'
        )
    ax.set_thetalim(-np.pi/2,np.pi/2)

    ax.quiver(
        thmesh[::rasterth,::rasterr],
        rmesh[::rasterth,::rasterr],
        cbx[::rasterth,::rasterr]*rmesh[::rasterth,::rasterr]**2,
        cby[::rasterth,::rasterr]*rmesh[::rasterth,::rasterr]**2,
        np.sign(cbr_lonslice)[::rasterth,::rasterr],
        cmap="bwr",
        units="dots",
        color="white",
        scale=1/20,
        pivot="middle",
    )
    ax.set_rlim(0,rmax)
#     ax.plot(np.radians(psp.lat),psp.radius.to("R_sun"),color="black")
    #plt.figure()
    # plt.pcolormesh(
    #     hbr.r_coords,
    #     hbr.theta_coords,
    #     hbr.data[0,:,:,0],cmap="bwr"
    #     )
    
from ipywidgets import interact
interact(do_lon_slice,
         idx=(0,len(cbr.phi_coords),10),
         rmax=(2,30,2),
         rasterth=(1,20,2),
         rasterr=(1,20,2)
        )

NameError: name 'cbr' is not defined

In [4]:
idx=0

cn = corona_model['rho']

cbr= corona_model['br']
cbt = corona_model['bp']
cbn = corona_model['bt']

cvr= corona_model['vr']
cvt = corona_model['vp']
cvn = corona_model['vt']

cn_lonslice = cn.data[idx,:,:,0] 

cbr_lonslice = cbr.data[idx,:,:,0]
cbt_lonslice = cbt.data[idx,:,:,0]
cbn_lonslice = cbn.data[idx,:,:,0]

cvr_lonslice = cvr.data[idx,:,:,0]
cvt_lonslice = cvt.data[idx,:,:,0]
cvn_lonslice = cvn.data[idx,:,:,0]

## Interpolate to same coords 
cbt_interpolated = interp2d(cbt.r_coords,
                            cbt.theta_coords,
                            cbt_lonslice)(cbr.r_coords,cbr.theta_coords)
cbn_interpolated = interp2d(cbn.r_coords,
                            cbn.theta_coords,
                            cbn_lonslice)(cbr.r_coords,cbr.theta_coords)

cb=np.sqrt(cbr_lonslice**2 + 
           cbt_interpolated**2 +
           cbn_interpolated**2)

NameError: name 'corona_model' is not defined

In [5]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(projection='polar')
rmesh,thmesh = np.meshgrid(cbr.r_coords,cbr.theta_coords)
ax.pcolormesh(
               cbr.theta_coords,
               cbr.r_coords,
               cbr_lonslice.T*rmesh.T**2,
               cmap="bwr",
              vmin = -6,vmax=6
)
ax.contour(
               cvr.theta_coords,
               cvr.r_coords,
               cvr_lonslice.T,
)

ax.set_thetalim(np.radians(-90),np.radians(90))
ax.grid()

NameError: name 'plt' is not defined